__N-Queens Problem__
> Implementation of the classical problem solution using an evolutionary computing approach

### Imports

In [11]:
import numpy as np
import random

### Constants

In [ ]:
RATE_CROSSOVER = 1
RATE_MUTATION = 0.8
SIZE_POPULATION = 20

# TODO...

In [1]:
'''
    Represents specific characteristics of individuals among a population.
    
    For this case it is:
    High level representation of one candidate solution that determines one configuration
    of queens positioning.
'''
phenotype = []

'''
    Represents the genetic composition that describes a phenotype characteristics.
    
    For this case it is:
    Lower level representation of phenotypes.
    
    i.e:
    Vector of n lines where each index represents a row and it's corresponding value represents a column.
    Each row/column combination determines where one queen is positioned. 
'''
genotype = []

### Utils

In [15]:
randomIndividual = lambda maxSize: random.sample(range(maxSize), maxSize)

def initPopulation(popSize: int, individualSize: int) -> list:
    population = []
    for _ in range(popSize):
        population.append(randomIndividual(individualSize))
    return population


foo = np.arange(0, 8, 1).tolist()
print(foo)
foo.remove(2)
foo

[0, 1, 2, 3, 4, 5, 6, 7]


[0, 1, 3, 4, 5, 6, 7]

## Handle table diagonals

In [ ]:
diagonals = {}

def getDiagonals(point: tuple, n: int) -> tuple:
    pI, pJ = point

    # Check if these diagonals have already been determined
    aux = diagonals.get(pI)
    cachedDiagonals = aux.get(pJ) if aux else None
    if cachedDiagonals:
        return cachedDiagonals

    rightDiag = []
    leftDiag = []

    # Right diagonal -> Upper Right
    i = pI - 1
    j = pJ + 1
    while i >= 0 and j < n:
        rightDiag.append( (i, j) )
        i -= 1
        j += 1

    # Right Diagonal -> Down Left
    i = pI + 1
    j = pJ - 1
    while i < n and j >= 0:
        rightDiag.append( (i, j) )
        i += 1
        j -= 1

    # Left Diagonal -> Upper Left
    i = pI - 1
    j = pJ - 1
    while i >= 0 and j >= 0:
        leftDiag.append( (i, j) )
        i -= 1
        j -= 1

    # Left Diagonal -> Down Right
    i = pI + 1
    j = pJ + 1
    while i < n and j < n:
        leftDiag.append( (i, j) )
        i += 1
        j += 1

    # Cache results
    pointDiagonals = (rightDiag, leftDiag)
    diagonals[i][j] = pointDiagonals
    
    return pointDiagonals


def isDiagonalMatch(point: tuple, diagonal: list) -> bool:
    row, col = point
    for diagPoint in diagonal:
        r, c = diagPoint
        isMatch = row == r and col == c
        if isMatch:
            return True
    return False

def isCheckmate(solution: list, i: int, j: int) -> bool:
    for solutionPoint in solution:
        row, col = solutionPoint

        isMe = i == row and j == col
        if not isMe:
            rowMatch = row == i
            if rowMatch:
                return True
            columnMatch = col == j
            if columnMatch:
                return True
        
        rightDiag, leftDiag = getDiagonals([], len(solution))
        if isDiagonalMatch((i, j), rightDiag) or isDiagonalMatch((i, j), leftDiag):
            return True
    return False

### Test

In [ ]:

def assertDiagonal(expectedDiag: list, realDiag: list) -> bool:
    # print(len(expectedDiag), len(realDiag))
    if len(expectedDiag) != len(realDiag):
        raise Exception('Length mismatch!')

    for expectedPoint in expectedDiag:
        i, j = expectedPoint
        isFound = False
        for realPoint in realDiag:
            if realPoint[0] == i and realPoint[1] == j:
                isFound = True
                break
        if not isFound:
            raise Exception('Missing point: ('+str(i)+', '+str(j)+')')

    return True

def runTest(point: tuple, expected: tuple, n: int) -> True:
    result = getDiagonals(point, n=n)
    print('[test] Result for point', point, ': ', result)
    
    matchedExpected = []
    matchedReal = []
    
    for i, expectedDiag in enumerate(expected):
        if not len(expectedDiag) or i in matchedExpected:
            continue
        
        isFound = False
        for j, realDiag in enumerate(result):
            if not len(realDiag) or j in matchedReal:
                continue
            if assertDiagonal(expectedDiag=expectedDiag, realDiag=realDiag):
                isFound = True
                matchedReal.append(j)
                break
        
        if isFound:
            matchedExpected.append(i)
            continue
    return True

N = 8
testPoints = [
    (
        (3, 3),
        N,
        (
            [(0, 0), (1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7)],
            [(0, 6), (1, 5), (2, 4), (4, 2), (5, 1), (6, 0)],
        ),
    ),
    (
        (0, 0),
        N,
        (
            [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7)],
            [],
        ),
    ),
    (
        (7, 0),
        N,
        (
            [(6, 1), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6), (0, 7)],
            [],
        ),
    ),
    (
        (7, 7),
        N,
        (
            [],
            [(6, 6), (5, 5), (4, 4), (3, 3), (2, 2), (1, 1), (0, 0)],
        ),
    ),
    (
        (0, 7),
        N,
        (
            [],
            [(1, 6), (2, 5), (3, 4), (4, 3), (5, 2), (6, 1), (7, 0)],
        ),
    ),
    (
        (4, 5),
        N,
        (
            [(2, 7), (3, 6), (5, 4), (6, 3), (7, 2)],
            [(0, 1), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7)],
        ),
    ),
]

for testPoint in testPoints:
    point, n, expected = testPoint
    print('[main] Testing: ', point)
    runTest(point=point, expected=expected, n=n)

## Algorithm

In [9]:

randomIndividual = lambda maxSize: random.sample(range(maxSize), maxSize)
def getInitialPopulation(popSize: int, individualSize: int) -> list:
    population = []
    for _ in range(popSize):
        population.append(randomIndividual(individualSize))
    return population

def fitnessFunction(individual: list) -> float:
    '''
        -- Evolutionary pressure --
        That is: How the environment conditions presurres species individuals to reinforce characteristics that
        grant necessary adaptiveness.

        For this case it means the least number of checkmates possible.

        This function returns how many checkmates are there in a given solution.
    '''

    nCheckmates = 0
    for i in individual:
        gene = individual[i]
        nCheckmates += int(isCheckmate(i, gene, individual))

    return nCheckmates

def selectParents(population: list) -> tuple:
    '''
        -- Natural Selection --
        Part of natural selection relative to selecting the best individuals to reproduce.

        For this case:
        Pick some random individuals from the population, rank them and make the crossover out of the 02 best fitted. 

        TODO: 2021-11-14 - Implement...
    '''

    return tuple()

def selectSurvivals() -> list:
    '''
        -- Natural Selection --
        Part of natural selection relative to selecting the best individuals to reproduce. 

        For this case:
        We will select the (n - 2) best solutions among all current n solutions. 

        TODO: 2021-11-14 - Implement...
    '''

    return []

def crossover(parents: tuple, probability: float) -> tuple:
    '''
        -- Recombination (as it is of 02 inputs it is called 'crossover') --
        That is: Hereditarius characteristics merging, where children characteristics are formed as result of
        parent characteristics merging.


        For this case it means generating a new solution by merging the 02 current best ones and, then permuting
        some genes. 
    '''

    willMutate = random.randint(0, 1) <= probability
    if not willMutate:
        return parents

    mom, dad = parents
    n = len(mom)
    pos = random.randint(1, n - 2)

    offspring = np.zeros(2, n)
    offspring[0] = mom[:pos]
    offspring[1] = dad[:pos]

    s1 = pos + 1;
    s2 = pos + 1;
    
    for i in range(n):
        check1 = False
        check2 = False
        
        for j in range(0, pos):
            check1 = dad[i] == offspring[0, j]
            check2 = mom[i] == offspring[1: j]
        
        if not check1:
            offspring[0][s1] = dad[i]
            s1 += 1
        if not check2:
            offspring[1][s2] = mom[i]
            s2 += 1

    return offspring[0], offspring[1]

def recombine(parents: tuple, crossoverRate: float = 1) -> list:
    '''
        -- Variation --
        TODO: 2021-11-14 - ADD Description
    '''
    return crossover(parents=parents, probability=crossoverRate)

def mutate(individual: list, probability: float, nGenes: int) -> list:
    '''
        -- Variation --
        Do 'nGenes' genetic mutations (i.e: permute 'nGenes' times (if mutation will really happen).
    '''

    willMutate = random.randint(0, 1) <= probability
    if not willMutate:
        return individual

    n = len(individual)
    for _ in range(nGenes):
        auxList = np.arange(0, n, 1).tolist()
        
        # Set 1st gene
        auxIdx = random.randint(0, n)
        gene1 = auxIdx
        auxList.remove(auxIdx)
        
        # Set 2nd gene
        auxIdx = random.randint(0, n - 1)
        gene2 = auxIdx
        auxValue = individual[gene2]
        
        # Permute
        individual[gene2] = individual[gene1]
        individual[gene1] = auxValue
    
    return individual

def evaluate(population: list) -> list:
    '''
        -- Natural selection step --
        Rank all individuals including the recent children added.
        TODO: 2021-11-14 - Implement...
    '''
    return []

def isOver():
    pass

RATE_CROSSOVER = 1
RATE_MUTATION = 0.8
SIZE_POPULATION = 20
def nQueens(
    n: int, popSize: int, maxGenerations: int,
    crossoverRate: float, mutationRate: float,
) -> list:
    
    population = getInitialPopulation(popSize=popSize, n=n)

    while not isOver():
        parents = selectParents(population)
        
        # Variate
        offspring = recombine(parents=parents, crossoverRate=crossoverRate)
        for i in range( len(offspring) ):
            offspring[i] = mutate(individual=offspring[i], probability=mutationRate, nGenes=2)
            population.append(offspring)
        
        population = evaluate(population)
        population = selectSurvivals(population)
    